In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from numpy.core.fromnumeric import size
from scipy.interpolate import griddata
from matplotlib.gridspec import GridSpec
import sys, os, subprocess
import json as json
from tqdm import tqdm
import pandas as pd
import argparse
from pathlib import Path
import seaborn as sns
import matplotlib.tri as tri
sys.path.append("../")
from libraries.functions import *
from libraries.particles import *
from libraries.exhumation import *
import plotly.graph_objects as go

/home/vturino/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
def get_trench_position_from_op(p, threshold = 2.7e7):
    if {"opc"}.issubset(p.columns):
        tr =  p.loc[(p['Points:0']< threshold) & (p["opc"] > 0.3) & (p["Points:1"] >=  p["Points:1"].max() - 2.e3),'Points:0'].min()
    else:
        tr =  p.loc[(p['Points:0']< threshold) & (p['op'] > 0.3) & (p["Points:1"] >=  p["Points:1"].max() - 2.e3),'Points:0'].min()
    return tr


In [3]:
file = {
    "models":[
      "kinematic_mu0.13_basalt7.5km_sed1km_cttV",
      "kinematic_mu0.13_basalt7.5km_sed1km_Vto50pc",
      "kinematic_mu0.13_basalt7.5km_sed1km_Vto0"
    ],
    "plot_folder":[
        "kinematic_mu0.13_basalt7.5km_sed1km_cttV",
        "kinematic_mu0.13_basalt7.5km_sed1km_Vto50pc",
        "kinematic_mu0.13_basalt7.5km_sed1km_Vto0"
    ],
    "head_lines": 23,
    "viz_lines": 23,
    "compositions":[
      "oc",
      "sed",
      "opc",
      "ecl"
    ],
    "track":[
      "oc",
      "sed",
      "ecl"
    ],
    "cutoff":[
      0,
      800000,
      0,
      750000
    ]
 }

csvs_loc = '/home/vturino/Vale_nas/exhumation/gz_outputs/'


compositions = file['compositions']
track = file['track']
tr = 0.5        

In [5]:
# For the cttV model, open the first timestep
model = file['models'][0]
plot_folder = file['plot_folder'][0]
gzip = f"{csvs_loc}{model}/particles/full.1.gzip"
initial = load_data(csvs_loc+model, '/particles/full.1.gzip', compositions, tr)

In [6]:
trench= get_trench_position_from_op(initial)

In [7]:
def collect_particles (trench:float, min_d: float, max_d: float, init):
    data = init[(init["Points:0"] < trench - min_d) & (init["Points:0"] > trench - max_d) & (init["opc"] == 0)]
    return data

In [8]:
dataset = collect_particles(trench, 0, 350.e3, initial)
dataset["lithology"] = [0]*len(dataset)


/tmp/ipykernel_866123/630335754.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["lithology"] = [0]*len(dataset)


In [9]:
for ind_c, c in enumerate(track):
    weight = ind_c + 1 
    dataset["lithology"] += weight * dataset[c].round()

dataset["lithology"] = dataset["lithology"].round().astype(int)

# sample 5% of the dataset
msk = np.random.rand(len(dataset)) <= 0.2
sample = dataset[msk]


/tmp/ipykernel_866123/3378002342.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["lithology"] += weight * dataset[c].round()
/tmp/ipykernel_866123/3378002342.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["lithology"] = dataset["lithology"].round().astype(int)


In [ ]:
# Now, for each model, we want to look at the last file "full.100.gzip" and find the particles with "id" that are:
# - contained in "sample"
# - contained in each model[i]/particles/full.100.gzip



